<a href="https://colab.research.google.com/github/trinchero-promemoria/image_captioning/blob/main/AGI_vitullo_captioning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Collegamento a Google Drive
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [ ]:
!pip install transformers


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 22.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.8/294.8 kB 35.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 70.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 64.2 MB/s eta 0:00:00


## Captioning immagine singola

In [ ]:
import requests
from PIL import Image
from transformers import BlipProcessor, BlipForConditionalGeneration

processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base").to("cuda")

img_url = 'IMAGE.jpg'
raw_image = Image.open(requests.get(img_url, stream=True).raw).convert('RGB')

# conditional image captioning
text = "a photography of"
inputs = processor(raw_image, text, return_tensors="pt").to("cuda")

out = model.generate(**inputs)
print(processor.decode(out[0], skip_special_tokens=True))
# >>> a photography of a woman and her dog

# unconditional image captioning
inputs = processor(raw_image, return_tensors="pt").to("cuda")

out = model.generate(**inputs)
print(processor.decode(out[0], skip_special_tokens=True))


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1260: UserWarning: Using the model-agnostic default `max_length` (=20) to control thegeneration length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


a photography of a group of men in uniform
a group of men in uniform walking down a street


## Captioning da csv

In [ ]:
from tqdm import tqdm  # Importa la libreria tqdm
import pandas as pd
import requests
from io import BytesIO
import torch
import os


def pred_from_csv(csv_path: str):
  processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
  model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base").to("cuda")

  # Carica il file CSV
  df = pd.read_csv(csv_path)

  # Crea una lista per memorizzare i risultati
  results = []

  # Crea una barra di avanzamento
  progress_bar = tqdm(total=len(df), desc='Processing Images')

  for index, row in df.iterrows():
      image_url = row['media_url']  # Assume che la colonna con l'URL sia chiamata 'media_url'
      raw_image = Image.open(requests.get(image_url, stream=True).raw).convert('RGB')

      # image captioning
      inputs = processor(raw_image, return_tensors="pt").to("cuda")

      out = model.generate(**inputs, max_new_tokens=50)

      # Aggiungi le informazioni a results
      results.append({
          'image_url': image_url,
          'caption': processor.decode(out[0], skip_special_tokens=True)
      })

      # Aggiorna la barra di avanzamento
      progress_bar.update(1)

  # Chiudi la barra di avanzamento
  progress_bar.close()

  # Crea un DataFrame dalle informazioni raccolte
  result_df = pd.DataFrame(results)

  # Genera automaticamente il nome del file di output
  csv_dir, csv_filename = os.path.split(csv_path)
  output_csv_path = os.path.join(csv_dir, f"captioning_output_{os.path.splitext(csv_filename)[0]}.csv")

  # Salva il DataFrame in un nuovo file CSV se è stato fornito un percorso di output
  if output_csv_path is not None:
      result_df.to_csv(output_csv_path, index=False)

  return result_df


In [ ]:
pred_from_csv('FILE.csv')


Processing Images: 100%|██████████| 826/826 [28:50<00:00,  2.09s/it]


,image_url,caption
0,https://agi.caveaudigitale.com/Archivio%20Vitu...,a large crowd of people in a large room
1,https://agi.caveaudigitale.com/Archivio%20Vitu...,two men in uniform talking to each other men
2,https://agi.caveaudigitale.com/Archivio%20Vitu...,a group of people standing in front of a building
3,https://agi.caveaudigitale.com/Archivio%20Vitu...,a man is loading a box on a truck
4,https://agi.caveaudigitale.com/Archivio%20Vitu...,a group of people standing in front of a building
...,...,...
821,https://agi.caveaudigitale.com/Archivio%20Vitu...,a group of men in uniform marching down a street
822,https://agi.caveaudigitale.com/Archivio%20Vitu...,a man and woman standing on a balcony
823,https://agi.caveaudigitale.com/Archivio%20Vitu...,a man kneeling on the ground next to a statue
824,https://agi.caveaudigitale.com/Archivio%20Vitu...,a group of people walking down a street
